# Lab #8: Climate policy and the DICE model (Patrick J. Applegate, Gregory G. Garner, Richard B. Alley, and Klaus Keller)

After completing this exercise, you should be able to

* describe what the social cost of carbon (SCC) is
* explain in broad terms what the DICE model is and how it works
* perform simple calculations with the DICE model
* describe how uncertainties in the climate sensitivity affect the present-day social cost of carbon

## Tutorial

In [ ]:
lab8_done.R
# Patrick Applegate, patrick.applegate@psu.edu; Greg Garner, ggg121@psu.edu
# 
# Optimizes the DICE model to produce a plausible climate-economic trajectory
# and performs a Monte Carlo experiment to evaluate the effects of uncertainty
# in the climate sensitivity on the present-day social cost of carbon.  
# Mise en place.  
rm(list = ls())
graphics.off()
# Load the DICE model.  
source("dice.R")
# Create a new DICE object.  
my.dice <- dice.new()

In [ ]:
# Solve the DICE object for the optimal control policy.  
# NOTE: THIS STEP MAY TAKE A COUPLE OF MINUTES!  
dice.solve(my.dice)
# Set sensible names for the time-dependent variables to extract from DICE.  
names <- c("Time (yr)", 
           "Emissions (Gt CO_2/yr)", 
           "Atmospheric [CO_2] (ppm)", 
           "Global mean T anomaly (C)", 
           "Climate damages (10^12 $)", 
           "Social cost of carbon ($/t CO_2)")
# Make a place to store time-dependent output from optimized DICE.  
opt.output <- matrix(data = NA, nrow = length(my.dice$year), ncol = 6)
colnames(opt.output) <- names
# Put the time.dependent output from optimized DICE into opt.output.  
opt.output[, 1] <- my.dice$year
opt.output[, 2] <- my.dice$e
opt.output[, 3] <- my.dice$mat
opt.output[, 4] <- my.dice$tatm
opt.output[, 5] <- my.dice$damages
opt.output[, 6] <- my.dice$scc

In [ ]:
# Plot the time-dependent output from the optimized DICE object.  
pdf("lab8_plot1.pdf", width = 5, height = 8.5)
par(mfrow = c(5, 1))
plot(opt.output[, 1], opt.output[, 2], type = "l", bty = "n", xlab = names[1], 
     ylab = names[2])
plot(opt.output[, 1], opt.output[, 3], type = "l", bty = "n", xlab = names[1], 
     ylab = names[3])
plot(opt.output[, 1], opt.output[, 4], type = "l", bty = "n", xlab = names[1], 
     ylab = names[4])
plot(opt.output[, 1], opt.output[, 5], type = "l", bty = "n", xlab = names[1], 
     ylab = names[5])
plot(opt.output[, 1], opt.output[, 6], type = "l", bty = "n", xlab = names[1], 
     ylab = names[6])
dev.off()

### Running DICE in optimization mode and examining its time-varying output

In [ ]:
# Also extract the preset climate sensitivity and social cost of carbon
# from the DICE object.  
opt.t2xCO2 <- my.dice$t2xCO2
opt.scc <- opt.output[1, 6]
# Print some key quantities.  
print(sprintf("The assumed climate sensitivity for optimization is %2.2f C/doubling", 
              opt.t2xCO2))
print(sprintf("The optimized social cost of carbon in %d is $%4.2f/t CO_2", 
              opt.output[1, 1], opt.scc))

### Accounting for uncertainty in the climate sensitivity

In [ ]:
# Set some values.  
xs <- c(1, 6)       # climate sensitivities corresponding to the probabilities 
                    # in ps
ps <- c(0.05, (1- 0.1))
                    # (approximate) probabilities of the climate sensitivity
                    # being less than xs[1] or greater than xs[2], according 
                    # to IPCC AR5 WG1
n.trials <- 300     # number of Monte Carlo trials
# Define a function for matching the lognormal distribution to two (or more)
# tie points.  
lnorm.rmse <- function(dist.params, xs, ps) {
  # dist.params, vector of meanlog and sdlog values; 
  # see help(dlnorm)
  # xs, vector of values of the distributed variable to match
  # ps, probabilities of the values in xs
  logmu <- dist.params[1]
  logsigma <- dist.params[2]
  trial.xs <- qlnorm(ps, logmu, logsigma)
  rmse <- sqrt(mean((xs- trial.xs)^ 2))
  return(rmse)
}
# Identify the parameters of the lognormal distribution that produce the best
# match to the IPCC's statements about climate sensitivity.  
lnorm.optim <- optim(log(c(2.9, 1.5)), lnorm.rmse, gr = NULL, xs = xs, ps = ps, 
                     method = "L-BFGS-B", lower = c(0, 0), upper = c(Inf, Inf))
# Generate a vector of climate sensitivity values.  
set.seed(1)
tx2CO2s <- rlnorm(n.trials, meanlog = lnorm.optim$par[1], 
                  sdlog = lnorm.optim$par[2])

## Sample Script

## Exercise